Copyright chriskeraly

Copyright (c) 2019 Lumerical Inc. 

## Imports
General purpose imports

In [ ]:
import os
import numpy as np
import scipy as sp

Import Lumerical

In [ ]:
import importlib.util
#default path for current release 
spec_lin = importlib.util.spec_from_file_location('lumapi', "/usr/local/ansys_inc/Lumerical/2023_r2/api/python/lumapi.py")
#Functions that perform the actual loading
lumapi = importlib.util.module_from_spec(spec_lin)
spec_lin.loader.exec_module(lumapi)

Optimization specific imports

In [ ]:
from lumopt.utilities.wavelengths import Wavelengths
from lumopt.geometries.polygon import FunctionDefinedPolygon
from lumopt.figures_of_merit.modematch import ModeMatch
from lumopt.optimizers.generic_optimizers import ScipyOptimizers
from lumopt.optimization import Optimization

## Define base simulation

In [ ]:
base_script = os.path.join(os.path.dirname(__file__), 'splitter_base_TE_modematch.lsf')

## Define spectral range
Global wavelength/frequency range for all the simulations

In [ ]:
wavelengths = Wavelengths(start = 1300e-9, stop = 1800e-9, points = 21)

## Define optimizable geometry
The class FunctionDefinedPolygon needs a parameterized Polygon (with points ordered in a counter-clockwise direction). Here the geometry is defined by 10 parameters defining the knots of a spline, and the resulting Polygon has 200 edges, making it quite smooth.

In [ ]:
initial_points_x = np.linspace(-1.0e-6, 1.0e-6, 10)
initial_points_y = np.linspace(0.25e-6, 0.6e-6, initial_points_x.size)
def taper_splitter(params = initial_points_y):
    ''' Defines a taper where the paramaters are the y coordinates of the nodes of a cubic spline. '''
    points_x = np.concatenate(([initial_points_x.min() - 0.01e-6], initial_points_x, [initial_points_x.max() + 0.01e-6]))
    points_y = np.concatenate(([initial_points_y.min()], params, [initial_points_y.max()]))
    n_interpolation_points = 100
    polygon_points_x = np.linspace(min(points_x), max(points_x), n_interpolation_points)
    interpolator = sp.interpolate.interp1d(points_x, points_y, kind = 'cubic')
    polygon_points_y = interpolator(polygon_points_x)
    polygon_points_up = [(x, y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points_down = [(x, -y) for x, y in zip(polygon_points_x, polygon_points_y)]
    polygon_points = np.array(polygon_points_up[::-1] + polygon_points_down)
    return polygon_points

The geometry will pass on the bounds and initial parameters to the optimizer.

In [ ]:
bounds = [(0.2e-6, 0.8e-6)] * initial_points_y.size

The permittivity of the material making the optimizable geometry and the permittivity of the material surrounding it must be defined. Since this is a 2D simulation, the depth has no importance. The edge precision defines the discretization of the edges forming the optimizable polygon. It should be set such there are at least a few points per mesh cell. An effective index of 2.8 is user to simulate a 2D slab of 220 nm thickness.

In [ ]:
geometry = FunctionDefinedPolygon(func = taper_splitter, initial_params = initial_points_y, bounds = bounds, z = 0.0, depth = 220e-9, eps_out = 1.44 ** 2, eps_in = 2.8 ** 2, edge_precision = 5, dx = 1e-9)

## Define figure of merit
The base simulation script defines a field monitor named 'fom' at the point where we want to modematch to the 3rd mode (fundamental TE mode).

In [ ]:
fom = ModeMatch(monitor_name = 'fom', mode_number = 2, direction = 'Forward', multi_freq_src = True, target_T_fwd = lambda wl: np.ones(wl.size), norm_p = 1)

## Define optimization algorithm
This will run Scipy's implementation of the L-BFGS-B algoithm for at least 40 iterations. Since the variables are on the order of 1e-6, thery are scale up to be on the order of 1.

In [ ]:
optimizer = ScipyOptimizers(max_iter = 30, method = 'L-BFGS-B', scaling_factor = 1e6, pgtol = 1e-5)

## Put everything together

In [ ]:
opt = Optimization(base_script = base_script, wavelengths = wavelengths, fom = fom, geometry = geometry, optimizer = optimizer, hide_fdtd_cad = False, use_deps = True)

## Run the optimizer

In [ ]:
opt.run()